# Web Scraping: Selenium

A menudo, los datos están disponibles públicamente para nosotros, pero no en una forma que sea fácilmente utilizable. Ahí es donde entra en juego el web scraping, podemos usar web scraping para obtener nuestros datos deseados en un formato conveniente que luego se puede usar. a continuación, mostraré cómo se puede extraer información de interés de un sitio web usando el paquete Selenium en Python. Selenium nos permite manejar una ventana del navegador e interactuar con el sitio web mediante programación. 

Selenium también tiene varios métodos que facilitan la extracción de datos.
En este Jupyter Notebook vamos a usar Python 3 en Windows.

En primer lugar, tendremos que descargar un controlador.

Usaremos ChromeDriver para Google Chrome. Para obtener una lista completa de controladores y plataformas compatibles, consulte [Selenium](https://www.selenium.dev/downloads/). Si desea utilizar Google Chrome, diríjase a [chrome](https://chromedriver.chromium.org/) y descargue el controlador que corresponde a su versión actual de Google Chrome.

Como saber cual es la version de chrome que utilizo simple utilizamos pegamos el siguiente enlace en la barra de chrome chrome://settings/help

Antes de comenzar se preguntaran si ya se BeautifulSoup cual es la diferencia con Selenium.

A diferencia BeautifulSoup, Selenium no trabaja con el texto fuente en HTML de la web en cuestión, sino que carga la página en un navegador sin interfaz de usuario. El navegador interpreta entonces el código fuente de la página y crea, a partir de él, un Document Object Model (modelo de objetos de documento o DOM). Esta interfaz estandarizada permite poner a prueba las interacciones de los usuarios. De esta forma se consigue, por ejemplo, simular clics y rellenar formularios automáticamente. Los cambios en la web que resultan de dichas acciones se reflejan en el DOM. La estructura del proceso de web scraping con Selenium es la siguiente:

URL → Solicitud HTTP → HTML → Selenium → DOM



## Comencemos importando las bibliotecas que usaremos:

In [1]:
from selenium import webdriver
import urllib3 # urllib3 es un cliente HTTP potente y fácil de usar para Python.
import re # Expresiones regulares 
import time
import pandas as pd

In [2]:
from selenium.webdriver.common.by import By

El objeto driver es con el que trabajaremos a partir de ahora

In [3]:
# especificamos el path hasta nuestro driver recién descargado:
chrome_driver_path = 'chromedriver.exe'
options  = webdriver.ChromeOptions()

In [4]:
# Creamos el driver con el que nos vamos a manejar en la sesión de scrapeo:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

C:\Users\debor\AppData\Local\Temp\ipykernel_13628\2445550838.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)


In [ ]:
# driver = webdriver.Firefox(executable_path='geckodriver.exe')

# from selenium.webdriver.firefox.options import Options
# options = Options()
# options.binary_location = "C:\Program Files\Mozilla Firefox\firefox.exe"
# driver = webdriver.Firefox(executable_path='geckodriver', options=options)
# driver.get('http://google.com/')

In [5]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://insolvencyinsider.ca/filing/'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)

Ahora si queremos hacer click en el boton de "Load more"..

Selenium proporciona varios métodos para localizar elementos en la página web. Usaremos el método find_element("xpath",) para crear un objeto de botón, con el que luego podremos interactuar:

/html/body/div[2]/div/main/div/div/div/button

In [6]:
loadMore = driver.find_element("xpath","//*[@id='content']/button")

In [7]:
print(loadMore)

<selenium.webdriver.remote.webelement.WebElement (session="9f71e614a63eb8359b9a34051da59884", element="cb02b591-02b0-4d70-beef-06f30fee4bb4")>


Antes de continuar, necesitaremos saber cuántas páginas hay para saber cuántas veces debemos hacer clic en el botón. Necesitaremos una forma de extraer el código fuente del sitio web. Afortunadamente, este proceso es relativamente sencillo con las bibliotecas urllib3 y re.

In [8]:
url = "https://insolvencyinsider.ca/filing/"
http = urllib3.PoolManager()
r = http.request("GET", url)
text = str(r.data)


```text``` ahora es una cadena. Ahora, necesitamos una forma de extraer total_pages de nuestra cadena de texto. Imprima texto para ver cómo podemos extraerlo usando RegEx con el paquete re. Podemos totalizar_páginas así:

In [9]:
totalPagesObj = re.search(pattern='"total_pages":\d+', string=text)
totalPagesStr = totalPagesObj.group(0)
totalPages = int((re.search(pattern="\d+", string=totalPagesStr)).group(0))

El método de búsqueda toma un patrón y una cadena. En este caso nuestro patrón es '"total_pages":\d+' . Si no está familiarizado con RegEx, todo esto significa que estamos buscando la cadena "total_pages": con dos o más dígitos después de los dos puntos. \d se refiere a un dígito entre 0 y 9, mientras que + indica que Python debe buscar una o más de las expresiones regulares anteriores. Puedes leer más sobre el paquete re aquí. El método search() devuelve un objeto Match. re proporciona el método group() que devuelve uno o más subgrupos de la coincidencia. Pasamos 0 como argumento para indicar que queremos el parche completo. La tercera línea simplemente extrae el entero correspondiente a total_pages de la cadena.

In [10]:
print(totalPagesObj)
print(totalPagesStr)
print(totalPages)

<re.Match object; span=(108880, 108897), match='"total_pages":106'>
"total_pages":106
106


Con eso completo, ahora podemos cargar todas las páginas de Insolvency Insider. Podemos hacer clic en el botón Cargar más accediendo al método click() del objeto. Esperamos tres segundos entre clics para no sobrecargar el sitio web.

Recuerde que el total de las páginas son 88 pero comenzamos en 0 asi que es 88-1

In [11]:
for _ in range(10):
    loadMore.click()
    time.sleep(3)

In [ ]:
for i in range(totalPages-1):
    loadMore.click()
    time.sleep(1)

Una vez que ejecute esto, debería ver que se hace clic en el botón Cargar más y que se cargan las páginas restantes.
Una vez que se carga cada página, podemos comenzar a raspar el contenido. Ahora, eliminar ciertos elementos como el nombre de presentación, la fecha y la hiperreferencia es bastante sencillo. Podemos usar los métodos find_elements() y find_element('xpath',) de Selenium (importante la ```s``` extra después de element):

- filing-name
- filing-date
- //*[@id='content']/div[2]/div/div[1]/h3/a

In [12]:
filingNamesElements = driver.find_elements(By.CLASS_NAME,"filing-name")

filingDateElements = driver.find_elements(By.CLASS_NAME,"filing-date")

filingHrefElements = driver.find_elements('xpath',"//*[@id='content']/div[2]/div/div[1]/h3/a")

In [13]:
len(filingNamesElements)

110

También nos gustaría conocer los metadatos de presentación, es decir, el tipo de archivo, el sector de la empresa y el lugar en la que operan. Extraer estos datos requiere un poco más de trabajo.

//*[@id='content']/div[2]/div[%d]/div[2]/div[1]

In [19]:
filingMetas = []
for i in range(len(filingNamesElements) + 1):
    filingMetai = driver.find_elements('xpath',("//*[@id='content']/div[2]/div[%d]/div[2]/div[1]" %(i)))
    # filingMetai = driver.find_elements(By.CLASS_NAME,'filing-meta')
    for element in filingMetai:
        filingMetaTexti = element.text
        filingMetas.append(filingMetaTexti)

De cada elemento de la presentación de Metas podemos extraer el tipo de presentación, la industria y la provincia, así:

********

In [22]:
metaDict = {"Filing Type": [], "Industry": [], "Province": []}
for filing in filingMetas:
    filingSplit = filing.split("\n")
  
    for item in filingSplit:
        itemSplit = item.split(":")

        
        if itemSplit[0] == "Filing Type":
            metaDict["Filing Type"].append(itemSplit[1])
        elif itemSplit[0] == "Industry":
            metaDict["Industry"].append(itemSplit[1])
        elif itemSplit[0] == "Province":
            metaDict["Province"].append(itemSplit[1])
            
    if "Filing Type" not in filing:
        metaDict["Filing Type"].append("NA")
    elif "Industry" not in filing:
        metaDict["Industry"].append("NA")
    elif "Province" not in filing:
        metaDict["Province"].append("NA")

In [23]:
for key in metaDict:
    print(len(metaDict[key]))

110
110
110


*********

Ahora, todavía tenemos que poner nuestros nombres y fechas de presentación en las listas. Hacemos esto agregando el texto de cada elemento a una lista usando el método text() de antes:

In [24]:
filingName = []
filingDate = []
filingLink = []
# para cada elemento en la lista de elementos de nombre de archivo, agrega el
# texto del elemento a la lista de nombres de archivo.
for element in filingNamesElements:
    filingName.append(element.text)
# para cada elemento en la lista de elementos de la fecha de presentación, agrega el
# texto del elemento a la lista de fechas de presentación.
for element in filingDateElements:
    filingDate.append(element.text)
for link in filingHrefElements:
    if link.get_attribute("href"):
        filingLink.append(link.get_attribute("href"))

Una vez que tengamos eso, estamos listos para poner todo en un diccionario y luego crear un DataFrame de pandas:

In [25]:
# Crea un diccionario final con nombres y fechas de archivo.
fullDict = {
    "Filing Name": filingName,
    "Filing Date": filingDate, 
    "Filing Type": metaDict["Filing Type"],
    "Industry": metaDict["Industry"],
    "Province": metaDict["Province"],
    "Link": filingLink
}
# Crea un DataFrame.
df = pd.DataFrame(fullDict)
df["Filing Date"] = pd.to_datetime(df["Filing Date"], infer_datetime_format=True)

In [26]:
df

,Filing Name,Filing Date,Filing Type,Industry,Province,Link
0,Manitoba Clinic Medical Corporation (“Medco”),2022-12-06,CCAA,Healthcare,Manitoba,https://insolvencyinsider.ca/filing/manitoba-c...
1,Nilex USA Inc. (“Nilex USA”),2022-11-24,NOI,Construction,Alberta,https://insolvencyinsider.ca/filing/nilex-usa-...
2,Lightbox Enteprises Ltd. dba Dutch Love Cannabis,2022-11-24,CCAA,Cannabis,British Columbia,https://insolvencyinsider.ca/filing/lightbox-e...
3,Telewizja Polska Canada Inc. (AKA Polish Telev...,2022-11-23,Bankruptcy,Media,Ontario,https://insolvencyinsider.ca/filing/telewizja-...
4,1138969 Ontario Inc. (“113”),2022-11-23,CCAA,Professional Services,Ontario,https://insolvencyinsider.ca/filing/1138969-on...
...,...,...,...,...,...,...
105,Home Solutions Corporation,2022-05-20,Bankruptcy,Manufacturing,Alberta,https://insolvencyinsider.ca/filing/home-solut...
106,Williams Telecommunications Corp.,2022-05-19,Receivership,Distribution,Ontario,https://insolvencyinsider.ca/filing/williams-t...
107,Terra Nova Mechanical Ltd.,2022-05-19,Receivership,Automotive,Alberta,https://insolvencyinsider.ca/filing/terra-nova...
108,Modern Acrylic Systems Ltd.,2022-05-18,Receivership,Professional Services,Manitoba,https://insolvencyinsider.ca/filing/modern-acr...


------------------------

# Ahora algo más visual

In [29]:
driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)

C:\Users\debor\AppData\Local\Temp\ipykernel_13628\3395105876.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = chrome_driver_path, options = options)


In [30]:
# indicamos la URL de la página web a la que queremos acceder:
url = 'https://www.filmaffinity.com/es/main.html'
# el objeto driver nos va a permitir alterar el estado del la página
driver.get(url)

La página de Filmaffinity se ha abierto

Pero....

Nos hemos encontrado con un pop-up que nos pide aceptar cookies

1. Buscamos el botón
2. Hacemos click en el botón

Vamos a quitar el boton para seguir

In [38]:
elements_by_tag = driver.find_elements(By.TAG_NAME,'button')
elements_by_class_name = driver.find_elements(By.CLASS_NAME,'css-v43ltw')
element_by_xpath = driver.find_element("xpath",'/html/body/div[1]/div/div/div/div[2]/div/button[2]')

In [33]:
elements_by_tag

[<selenium.webdriver.remote.webelement.WebElement (session="0cd8219df28f89b6af6fd26dc6d64876", element="ec97224a-cd9b-40cc-8b3f-ba948cbeb28c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0cd8219df28f89b6af6fd26dc6d64876", element="6634f24d-4cc5-4847-93bf-d0df90cbdd26")>,
 <selenium.webdriver.remote.webelement.WebElement (session="0cd8219df28f89b6af6fd26dc6d64876", element="c41770a8-f81b-4cbe-9baa-4500f337a8dd")>]

In [34]:
elements_by_class_name

[<selenium.webdriver.remote.webelement.WebElement (session="0cd8219df28f89b6af6fd26dc6d64876", element="c41770a8-f81b-4cbe-9baa-4500f337a8dd")>]

In [39]:
element_by_xpath

<selenium.webdriver.remote.webelement.WebElement (session="0cd8219df28f89b6af6fd26dc6d64876", element="c41770a8-f81b-4cbe-9baa-4500f337a8dd")>

Una vez tenemos los elementos podemos hacer varias cosas con ellos

Podemos extraer todos los atributos que tenga

In [35]:
dir(element_by_xpath)
# obtenemos todos sus métodos y atributos:

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_execute',
 '_id',
 '_parent',
 '_upload',
 'accessible_name',
 'aria_role',
 'clear',
 'click',
 'find_element',
 'find_elements',
 'get_attribute',
 'get_dom_attribute',
 'get_property',
 'id',
 'is_displayed',
 'is_enabled',
 'is_selected',
 'location',
 'location_once_scrolled_into_view',
 'parent',
 'rect',
 'screenshot',
 'screenshot_as_base64',
 'screenshot_as_png',
 'send_keys',
 'shadow_root',
 'size',
 'submit',
 'tag_name',
 'text',
 'value_of_css_property']

Podemos evaluar que tipo de elemento es (tag)

In [36]:
element_by_xpath.tag_name

'button'

Podemos sacar el valor que tiene (el texto)

In [40]:
element_by_xpath.text

'ACEPTO'

In [41]:
for i in range(0,len(elements_by_tag)):
    print(elements_by_tag[i].text)

socios
MÁS OPCIONES
ACEPTO


Incluso podemos guardar una imagen del elemento

In [42]:
type(element_by_xpath)
# Vemos que es tipo 'WebElement' y en la documentación podremos encontrar sus métodos

selenium.webdriver.remote.webelement.WebElement

In [43]:
# guardamos como 'mi_imagen.png' la imagen asociada al xpath
element_by_xpath.screenshot('mi_imagen.png')

True

Evaluamos que elementos hemos encontrado por el tag:

In [44]:
for index, element in enumerate(elements_by_tag):
    print('Elemento:', index)
    print('Texto del elemento',index, 'es', element.text)
    print('El tag del elemento',index, 'es', element.tag_name)
    element.screenshot('mi_imagen'+str(index)+'.png')

Elemento: 0
Texto del elemento 0 es socios
El tag del elemento 0 es button
Elemento: 1
Texto del elemento 1 es MÁS OPCIONES
El tag del elemento 1 es button
Elemento: 2
Texto del elemento 2 es ACEPTO
El tag del elemento 2 es button


Basta de tonterias seguimos

Instanciamos el elemento del tag [2] en la variable boton aceptar

In [45]:
# boton_aceptar = elements_by_tag[2]
boton_aceptar = element_by_xpath

Si el elemento es interactivo podremos hacer más cosas además de las anteriores. Por ejemplo: hacer click

In [46]:
boton_aceptar.click()

Buscamos una película por título

In [47]:
from selenium.webdriver.common.keys import Keys

In [60]:

buscador = driver.find_element("id",'top-search-input')

In [ ]:
buscador = driver.find_element("xpath",'/html/body/div[2]/div[1]/div/div[2]/form/div/input')

In [61]:
buscador.send_keys('Minions')

In [50]:
buscador.clear()

In [62]:
# una vez escrita la búsqueda deberíamos poder activarla:
buscador.send_keys(Keys.ENTER)

In [58]:
# volvemos a la página anterior
driver.back()

### Vamos a buscar todas las películas que se estrenan el próximo viernes

1. Cogemos los containers que hay en la zona lateral

In [78]:
menu_lateral = driver.find_element('id','lsmenu')
menu_lateral

<selenium.webdriver.remote.webelement.WebElement (session="0cd8219df28f89b6af6fd26dc6d64876", element="2fa6da2e-6045-4d70-b17f-0e5167ba799e")>

In [80]:
mis_secciones = menu_lateral.find_elements(By.TAG_NAME,'a')

In [66]:
for seccion in mis_secciones:
    print(seccion.text)

Votar los tours
Iniciar sesión
Registrarse
Películas en cartelera
Cines España
Próximos estrenos
Estrenos Blu-ray venta
Próximos Blu-ray venta
Ya para alquilar
Próximamente alquiler
Netflix
Netflix (próx)
Amazon Prime
Movistar Plus+
Movistar Plus+ (próx)
HBO Max
Disney+
Filmin
Atlàntida
Apple TV+
Rakuten TV
Estrenos USA
Estrenos Reino Unido
Estrenos Francia
Todos los rankings
El Top 1000 de FA
Taquilla
Trailers
Últimos trailers
Últimas críticas users
Qué dice la crítica
Películas por temas
Sagas & Franquicias
Series de actualidad
Top series
Vota Series de TV
Top Filmaffinity
Lo mejor del TOP
Top Estrenos
Top Estrenos Blu-ray
Ranking de listas
Premios y Festivales
Todos los Oscar
Resumen 2021
Acerca de FA
Contacto


2. Vemos con cuál nos tenemos que quedar

In [81]:
for a in mis_secciones:
    if a.text == 'Próximos estrenos':
        a.click()
        break

Accedemos al container central, en el que aparecen los estrenos por semana que queremos ver, exactamente igual que hemos hecho antes

In [82]:
cajon_central = driver.find_elements('id','main-wrapper-rdcat')

In [69]:
type(cajon_central)

list

In [70]:
for semana in cajon_central:
    print(semana.find_element(By.TAG_NAME,'div').text)
    print(semana.find_element(By.TAG_NAME,'div').get_attribute('id'))

16 de diciembre de 2022
2022-12-16
21 de diciembre de 2022
2022-12-21
23 de diciembre de 2022
2022-12-23
28 de diciembre de 2022
2022-12-28
4 de enero de 2023
2023-01-04
13 de enero de 2023
2023-01-13
20 de enero de 2023
2023-01-20
27 de enero de 2023
2023-01-27


In [83]:
for semana in cajon_central:
    fecha = semana.find_element(By.TAG_NAME,'div').get_attribute('id')
    if fecha == '2022-12-21':
        break

Buscamos cómo acceder a las películas

In [84]:
semana

<selenium.webdriver.remote.webelement.WebElement (session="0cd8219df28f89b6af6fd26dc6d64876", element="70ab3161-9616-4717-9ed9-0bd16f52b1d7")>

In [73]:
caratulas = semana.find_elements(By.CLASS_NAME,'movie-card')
lista_pelis = []
for peli in caratulas:
    lista_pelis.append(peli.find_element(By.TAG_NAME,'a').get_attribute('href'))

In [74]:
lista_pelis

['https://www.filmaffinity.com/es/film542141.html',
 'https://www.filmaffinity.com/es/film732443.html',
 'https://www.filmaffinity.com/es/film893654.html',
 'https://www.filmaffinity.com/es/film747706.html',
 'https://www.filmaffinity.com/es/film582272.html',
 'https://www.filmaffinity.com/es/film403812.html']

Una vez tenemos todas las urls vamos a ver qué hacemos con cada una de ellas

In [85]:
# Accedemos a la página de la primera pelicula
driver.get(lista_pelis[0])

Vamos a ver el proceso que deberíamos hacer con cada una de las películas:

In [87]:
driver.get("https://www.filmaffinity.com/es/film873946.html")

1. Sacamos toda la información que nos interesa

In [88]:
# titulo, nota, numero de votos y ficha técnica
titulo = driver.find_element("xpath",'/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span').text
nota = driver.find_element("xpath",'/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[1]').text
votos = driver.find_element("xpath",'/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]/span').text
ficha = driver.find_element("xpath",'/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')

In [89]:
titulo,nota,votos,ficha

('After. En mil pedazos',
 '3,5',
 '1.377',
 <selenium.webdriver.remote.webelement.WebElement (session="0cd8219df28f89b6af6fd26dc6d64876", element="6a022df7-6c5a-4d9e-83b4-710fcbabe99a")>)

2. Creamos una lista a partir de la ficha técnica

In [92]:
# Los nombres estan con tag  = 'dt' y los valores con 'dd'
ficha_names = []
ficha_values = []

for name in ficha.find_elements(By.TAG_NAME,'dt'):
    ficha_names.append(name.text)
for value in ficha.find_elements(By.TAG_NAME,'dd'):
    ficha_values.append(value.text)

In [91]:
ficha_names,ficha_values

(['Título original',
  '',
  'Año',
  'Duración',
  'País',
  'Dirección',
  'Guion',
  'Fotografía',
  'Reparto',
  'Compañías',
  'Género',
  'Grupos',
  'Sinopsis'],
 ['After We Collidedaka',
  '',
  '2020',
  '105 min.',
  ' Estados Unidos',
  'Roger Kumble',
  'Mario Celaya. Novela: Anna Todd',
  'Larry Reibman',
  'Josephine Langford, Hero Fiennes-Tiffin, Dylan Sprouse, Selma Blair, Candice Accola, Charlie Weber, Samuel Larsen, Louise Lombard, Dylan Arnold, Inanna Sarkis, ver 11 más',
  'CalMaple, Frayed Pages Entertainment. Distribuidora: GEM Entertainment',
  'Romance. Drama | Secuela. Young Adult. Adolescencia',
  'After',
  "Ha pasado el tiempo, y Hardin (Hero Finnes Tiffin) todavía no se sabe si es realmente el chico profundo y reflexivo del que Tessa (Josephine Langford) se enamoró, o ha sido un extraño todo este tiempo. Ella quiere alejarse, pero no es tan fácil. Tessa se ha centrado en sus estudios y comienza a trabajar como becaria en Vance Publishing. Allí conoce a Trev

3. Creamos un dataframe con la info

In [93]:
columns = ['Titulo', 'Nota', 'Votos']
columns.extend(ficha_names)
len(columns)

16

In [94]:
values = [titulo, nota, votos]
values.extend(ficha_values)
len(values)

16

In [95]:
pd.DataFrame([values],columns=columns)

,Titulo,Nota,Votos,Título original,,Año,Duración,País,Dirección,Guion,Fotografía,Reparto,Compañías,Género,Grupos,Sinopsis
0,After. En mil pedazos,"3,5",1.377,After We Collidedaka,,2020,105 min.,Estados Unidos,Roger Kumble,Mario Celaya. Novela: Anna Todd,Larry Reibman,"Josephine Langford, Hero Fiennes-Tiffin, Dylan...","CalMaple, Frayed Pages Entertainment. Distribu...",Romance. Drama | Secuela. Young Adult. Adolesc...,After,"Ha pasado el tiempo, y Hardin (Hero Finnes Tif..."


Ahora vamos a crear una función que nos haga todo esto para cada una de las películas:

In [96]:
def sacar_info(driver):
    
    titulo = driver.find_element("xpath",'/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/h1/span').text
    try:
        nota = driver.find_element("xpath",'/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]').text
        votos = driver.find_element("xpath",'/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[2]/div[2]/div[1]/div[2]/div[2]').text
    except:
        nota = None
        votos = None
    ficha = driver.find_element("xpath",'/html/body/div[4]/table/tbody/tr/td[2]/div[1]/div[4]/div/div[3]/dl[1]')
    
    return titulo, nota, votos, ficha

def sacar_ficha(ficha):
    
    ficha_names = []
    ficha_values = []

    for name in ficha.find_elements(By.TAG_NAME,'dt'):
        ficha_names.append(name.text)
    for value in ficha.find_elements(By.TAG_NAME,'dd'):
        ficha_values.append(value.text)
        
    return ficha_names, ficha_values

def montar_df(ficha_names, ficha_values, titulo, nota, votos):
    
    columns = ['Titulo', 'Nota', 'Votos']
    columns.extend(ficha_names)
    values = [titulo, nota, votos]
    values.extend(ficha_values)
    
    return pd.DataFrame([values], columns = columns)
    
def nueva_pelicula(driver):
    
    titulo, nota, votos, ficha = sacar_info(driver)
    ficha_names, ficha_values = sacar_ficha(ficha)
    df_peli = montar_df(ficha_names, ficha_values, titulo, nota, votos)
    
    return df_peli


Vamos a ver cómo nos podemos mover entre ventanas del navegador

Abrir nueva ventana:

In [97]:
driver.execute_script('window.open("");')

Movernos a otra ventana

In [98]:
driver.switch_to.window(driver.window_handles[0])

Cerrar ventana

In [99]:
driver.close()

Una vez cerramos la ventana tenemos que indicarle a qué ventana tiene que ir

In [100]:
driver.switch_to.window(driver.window_handles[-1])

Sabiendo cómo podemos movernos por entre las ventanas y sabiendo cómo extraer de cada página toda la información que necesitamos vamos a crear nuestro dataframe:

In [101]:
# para abrir todos los links en lista_pelis
for link in lista_pelis:
    driver.execute_script('window.open("'+link+'");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)

In [102]:
# Creamos un dataframe con todas las pelis que se estrenan la próxima semana:
df_peliculas = pd.DataFrame()

for link in lista_pelis:
    driver.execute_script('window.open("");')
    driver.switch_to.window(driver.window_handles[-1])
    driver.get(link)
    nueva_peli = nueva_pelicula(driver)
    df_peliculas = df_peliculas.append(nueva_peli)

C:\Users\debor\AppData\Local\Temp\ipykernel_13628\2147100405.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_peliculas = df_peliculas.append(nueva_peli)
C:\Users\debor\AppData\Local\Temp\ipykernel_13628\2147100405.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_peliculas = df_peliculas.append(nueva_peli)
C:\Users\debor\AppData\Local\Temp\ipykernel_13628\2147100405.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_peliculas = df_peliculas.append(nueva_peli)
C:\Users\debor\AppData\Local\Temp\ipykernel_13628\2147100405.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_peliculas = df_peliculas.append(nueva_peli)
C:\U

In [103]:
df_peliculas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 0 to 0
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Titulo           6 non-null      object
 1   Nota             3 non-null      object
 2   Votos            3 non-null      object
 3   Título original  6 non-null      object
 4   Año              6 non-null      object
 5   Duración         5 non-null      object
 6   País             6 non-null      object
 7   Dirección        6 non-null      object
 8   Guion            6 non-null      object
 9   Música           5 non-null      object
 10  Fotografía       6 non-null      object
 11  Reparto          6 non-null      object
 12  Compañías        6 non-null      object
 13  Género           6 non-null      object
 14  Grupos           2 non-null      object
 15  Sinopsis         6 non-null      object
 16                   3 non-null      object
dtypes: object(17)
memory usage: 864.0+ byte

In [104]:
df_peliculas

,Titulo,Nota,Votos,Título original,Año,Duración,País,Dirección,Guion,Música,Fotografía,Reparto,Compañías,Género,Grupos,Sinopsis,
0,El gato con botas: El último deseo,None,None,Puss in Boots: The Last Wish,2022,100 min.,Estados Unidos,"Joel Crawford, Januel Mercado","Paul Fisher, Tommy Swerdlow. Historia: Tommy S...",Heitor Pereira,Animación,Animación,"DreamWorks Animation, Pacific Data Images (PDI...",Animación. Aventuras. Comedia | Gatos. Secuela,Shrek | DreamWorks Animation (Películas) | El ...,Secuela de 'El gato con botas' (2011). El Gato...,NaN
0,I Wanna Dance with Somebody,None,None,I Wanna Dance with Somebody,2022,NaN,Estados Unidos,Kasi Lemmons,Anthony McCarten,NaN,Barry Ackroyd,"Naomi Ackie, Ashton Sanders, Stanley Tucci, Cl...",Coproducción Estados Unidos-Reino Unido; TriSt...,Drama | Biográfico. Música,NaN,Biopic sobre la cantante Whitney Houston. Un r...,NaN
0,Broker,"6,6\n268\nvotos",268\nvotos,Brokeraka,2022,129 min.,Corea del Sur,Hirokazu Koreeda,Hirokazu Koreeda,Jung Jae-il,Hong Kyung-pyo,"Song Kang-ho, Gang Dong-won, Bae Doona, Lee Ji...",Zip Cinema,Drama,NaN,Una noche lluviosa una joven abandona a su beb...,
0,Ego,"6,2\n720\nvotos",720\nvotos,Pahanhautojaaka,2022,88 min.,Finlandia,Hanna Bergholm,Ilja Rautsi. Historia: Hanna Bergholm,Stein Berge Svendsen,Jarkko T. Laine,"Siiri Solalinna, Sophia Heikkilä, Jani Volanen...",Coproducción Finlandia-Suecia; Silva Mysterium...,Terror. Thriller | Familia. Gimnasia. Thriller...,NaN,Tinja es una joven gimnasta que intenta desesp...,
0,Eugénie Grandet,None,None,Eugénie Grandet,2021,103 min.,Francia,Marc Dugain,Marc Dugain. Novela: Honoré de Balzac,Jeremy Hababou,Gilles Porte,"César Domboy, Olivier Gourmet, Joséphine Japy,...","Featuristic Films, High Sea Production",Drama | Siglo XIX,Adaptaciones de Honoré de Balzac,Felix Grandet reina supremo en su modesta casa...,NaN
0,Dilo alto y fuerte,"5,7\n25\nvotos",25\nvotos,Haut et fortaka,2021,101 min.,Marruecos,Nabil Ayouch,Nabil Ayouch,"Mike Kourtzer, Fabien Kourtzer","Amine Messadi, Virginie Surdej","Ismail Adouab, Nouhaila Arif, Zineb Boujemaa, ...",Coproducción Marruecos-Francia; Ali n' Product...,Drama | Música. Hip Hop. Enseñanza,NaN,Anas es un exrapero que trabaja en un centro c...,


Ya tenemos un dataframe con todas las películas que se van a estrenar el próximo viernes